In [1]:
import pandas as pd
import numpy as np

In [2]:
# Load your base notebook (embeddings, df, helpers)
LIGHT_IMPORT = True
%run -i "Calc_Embeddings_Model Training.ipynb"

# quick sanity
print("df shape:", df.shape if 'df' in globals() else 'df missing')
print("have get_matches?", 'get_matches' in globals())

shape: (59946, 31)


You should consider upgrading via the 'C:\Users\veera\AppData\Local\Microsoft\WindowsApps\PythonSoftwareFoundation.Python.3.9_qbz5n2kfra8p0\python.exe -m pip install --upgrade pip' command.


QUERY user 1234 | age 31 | m straight | palo alto, california
------------------------------------------------------------------------------------------
hello! thanks for stopping by! i hope you enjoy yourself!  i am extremely financially responsible and nice! need a new kidney? i'll give you mine! if the doctor won't let us do that, i'll just buy you a new one! no expense is too much for you!  i definitely have skinny genes! rea...

=== Matches ===

user 44713 | age 23 | m straight | hayward, california | score=0.594
hey my name is jacob i'm a die hard raider! lol i work as a massage therapist at chiropractic offices and my personal clients i do some personal tranning generally weight loss, nutrition, and hypertrophy. i'm currently attending ohlone college working on a physical therapy assist...
------------------------------------------------------------------------------------------
user 10612 | age 28 | m gay | san francisco, california | score=0.564
high energy goofy nerd-jock-goo

In [5]:
# --- knobs (adjust later if you want) ---
CUPID_TOP_K   = 20
CUPID_POOL_K  = 500
CUPID_MAX_KM  = 50

def cupid_candidate_pool(user_id: int,pool_k: int = CUPID_POOL_K,max_km: float = CUPID_MAX_KM) -> pd.DataFrame:
    """
    Build the candidate pool using only hard gates:
      - orientation/sex as the user specifies (inside get_matches)
      - age range
      - location radius
    NOTE: No lifestyle gating here. We'll handle lifestyle softly during scoring.
    """
    cands = get_matches(user_id=user_id, k=pool_k,use_orientation=True,use_location=True, max_km=max_km,use_age=True,
        # all lifestyle OFF here -> soft handling later
        use_diet=False, use_drinks=False, use_smokes=False, use_drugs=False,
        drinks_strict=False, smokes_strict=False, drugs_strict=False,
        allow_missing=True).copy()

    # ensure a minimal set of columns is present (keep others if they exist)
    base_cols = ["user_id", "age", "sex", "orientation", "location"]
    for col in base_cols:
        if col not in cands.columns:
            cands[col] = np.nan

    # return with base cols first, then any extra columns that came along
    ordered = base_cols + [c for c in cands.columns if c not in base_cols]
    return cands[ordered]

In [ ]:
"""
sets a few simple knobs (CUPID_TOP_K, CUPID_POOL_K, CUPID_MAX_KM).
defines cupid_candidate_pool(...), which calls your get_matches with only hard gates (orientation, age, location). 
lifestyle is intentionally left out here, it will be soft later in the scoring step.
runs a quick smoke test so you can see the pool size and first 10 rows
"""

In [6]:
u = 14
pool = cupid_candidate_pool(user_id=u)
print("pool size:", len(pool))
pool.head(10)

Start pool: 5000 (max_candidates=5000)
After orientation: 2478
Unfiltered: 2478 | After location filter (≤50 km): 2457
After location (≤50 km): 2457
After age rule: 2225
pool size: 500


,user_id,age,sex,orientation,location,text_sim,diet_c,drinks_c,smokes_c,drugs_c,lat,lon
2,38159,32,m,straight,"san carlos, california",0.742732,,light,none,none,37.504936,-122.261823
7,40022,25,m,straight,"san francisco, california",0.736307,omnivore,heavy,light,,37.779259,-122.419329
9,25427,29,m,straight,"mountain view, california",0.735156,vegetarian,light,none,none,37.389389,-122.083210
10,3717,37,m,straight,"palo alto, california",0.734164,,,light,,37.444329,-122.159847
11,28563,29,m,straight,"san francisco, california",0.732910,,light,none,none,37.779259,-122.419329
12,39450,25,m,straight,"san francisco, california",0.732575,omnivore,,none,none,37.779259,-122.419329
16,23663,27,m,straight,"san mateo, california",0.730931,omnivore,light,none,none,37.496904,-122.333057
20,15046,36,m,straight,"san francisco, california",0.728997,omnivore,light,none,none,37.779259,-122.419329
21,33656,26,m,straight,"san francisco, california",0.728639,omnivore,light,none,none,37.779259,-122.419329
26,46695,31,m,straight,"palo alto, california",0.726020,vegetarian,light,none,none,37.444329,-122.159847
